In [120]:
import tensorflow as tf
import keras
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import random


In [121]:
def build_MLP(n_nodes=200,activation_func="relu",learning_rate=0.01,input_Shape=[28,28],output_shape=10):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(input_Shape[0],input_Shape[1])))
    model.add(keras.layers.Dense(n_nodes, activation=activation_func))
    model.add(keras.layers.Dense(output_shape, activation="softmax"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=["accuracy"])
    return model

def build_test_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    model.add(keras.layers.Dense(300, activation="relu"))
    model.add(keras.layers.Dense(100, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer_sgd=keras.optimizers.SGD(learning_rate=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer_sgd, metrics=["accuracy"])
    return model

In [126]:
fashion_mnist=keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full[0])
print(X_train_full[0]/255.0)

x_train,x_val,y_train,y_val=sklearn.model_selection.train_test_split(X_train_full/255.0,y_train_full,test_size=0.1,random_state=42)
print(x_train.shape)



(54000, 28, 28)


In [134]:
#hyper parameter tuning through random search
tests=5
n_nodes=np.arange(199)+1
activation_functions=["elu","relu","sigmoid"]
learning_rates=[0.0003,0.03]
used_parameters=[]
gained_scores=[]
early_stoping=keras.callbacks.EarlyStopping(patience=5)
for i in range(tests):
    print("search "+str(i+1))
    nodes=n_nodes[random.randint(0,len(n_nodes)-1)]
    activation=activation_functions[random.randint(0,len(activation_functions)-1)]
    learning_rate=random.uniform(learning_rates[0],learning_rates[1])
    used_parameters.append([nodes,activation,learning_rate])

    model=build_MLP(n_nodes=nodes,activation_func=activation,learning_rate=learning_rate)
    #model=build_test_model()
    history = model.fit(x_train, y_train, epochs=5,validation_data=(x_val, y_val),callbacks=[early_stoping])
    gained_scores.append(model.evaluate(X_test, y_test)[1])
print(used_parameters)
print(gained_scores)


search 1
Epoch 1/5
1688/1688 [==============================] - 14s 7ms/step - loss: 1.0515 - accuracy: 0.6982 - val_loss: 0.7235 - val_accuracy: 0.7567
Epoch 2/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.6501 - accuracy: 0.7762 - val_loss: 0.6090 - val_accuracy: 0.7845
Epoch 3/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5701 - accuracy: 0.8028 - val_loss: 0.5527 - val_accuracy: 0.8038
Epoch 4/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.5279 - accuracy: 0.8162 - val_loss: 0.5158 - val_accuracy: 0.8172
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 1.2219 - accuracy: 0.7409
search 2
Epoch 1/5
1688/1688 [==============================] - 11s 6ms/step - loss: 1.2022 - accuracy: 0.6804 - val_loss: 0.8256 - val_accuracy: 0.7417
Epoch 2/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.7173 - accuracy: 0.7676 - val_loss: 0.6579 - val_accuracy: 0.7772
Epoch 3/5
1688/1688 [